In [4]:
import os
import sys
import argparse
import logging

import numpy as np

from DataLoader import create_dataloader
from Detectors import create_detector
from Matchers import create_matcher

import ipyvolume as ipv
from visualizer.ipw_3d_helper import ipv_prepare, ipv_draw_pose_3d, ipv_draw_point_cloud, iPV3DVisualizer


In [5]:
from SFM import sfm

args = argparse.Namespace()
args.config = 'configs/llff_trex_superpoint_superglu.yaml'
args.logging = 'INFO'
logging.basicConfig(level=logging._nameToLevel[args.logging])

# Load config file
config = None
if args.config is not None:
    import yaml
    with open(args.config, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
else:
    logging.error("Config file is not specified.")
    sys.exit(-1)

# Create data loaders
dataloader = create_dataloader(config["dataset"])
# create detector
detector = create_detector(config["detector"])
# create matcher
matcher = create_matcher(config["matcher"])

pipeline = sfm.SFM(dataloader, detector, matcher, camera=dataloader.cam, config=config["sfm"])

# Run the pipeline
pipeline.reconstruct()

# visualize all views and point clouds
ipv_prepare(ipv)
Tcw_lst = []
for i, view in enumerate(pipeline.reconstructed_views):
    Tcw = np.eye(4)
    Tcw[:3, :3] = view.R
    Tcw[:3, 3] = view.t.T
    Tcw_lst.append(Tcw)
points_lst = pipeline.reconstructed_map_points

ipv_visualizer = iPV3DVisualizer()
ipv_visualizer.add_trajectory(Tcw_lst)
ipv_visualizer.add_point_cloud(points_lst, pt_size=8.0)
ipv_visualizer.show()



INFO:root:Sequence image loader config: 
INFO:root:{'root_path': '../data/trex', 'format': 'JPG', 'img_downscale_factor': 8, 'pose_recenter': False, 'name': 'llff'}


img_path_lst: ['DJI_20200223_163548_810.png', 'DJI_20200223_163551_210.png', 'DJI_20200223_163551_990.png', 'DJI_20200223_163553_010.png', 'DJI_20200223_163553_790.png', 'DJI_20200223_163554_630.png', 'DJI_20200223_163555_770.png', 'DJI_20200223_163556_700.png', 'DJI_20200223_163557_660.png', 'DJI_20200223_163558_500.png', 'DJI_20200223_163559_325.png', 'DJI_20200223_163600_150.png', 'DJI_20200223_163600_960.png', 'DJI_20200223_163601_770.png', 'DJI_20200223_163602_580.png', 'DJI_20200223_163603_540.png', 'DJI_20200223_163604_605.png', 'DJI_20200223_163605_490.png', 'DJI_20200223_163606_270.png', 'DJI_20200223_163607_050.png', 'DJI_20200223_163607_906.png', 'DJI_20200223_163608_715.png', 'DJI_20200223_163609_435.png', 'DJI_20200223_163610_185.png', 'DJI_20200223_163611_160.png', 'DJI_20200223_163614_400.png', 'DJI_20200223_163615_330.png', 'DJI_20200223_163616_140.png', 'DJI_20200223_163616_980.png', 'DJI_20200223_163617_790.png', 'DJI_20200223_163618_570.png', 'DJI_20200223_163619_411

INFO:root:SuperPoint detector config: 
INFO:root:{'descriptor_dim': 256, 'nms_radius': 4, 'keypoint_threshold': 0.005, 'max_keypoints': -1, 'remove_borders': 4, 'path': PosixPath('/home/hkust/fangchuan/codes/project4/python/Detectors/superpoint/superpoint_v1.pth'), 'cuda': 1, 'name': 'SuperPointDetector'}
INFO:root:creating SuperPoint detector...
INFO:root:SuperGlue matcher config: 
INFO:root:{'descriptor_dim': 256, 'weights': 'indoor', 'keypoint_encoder': [32, 64, 128, 256], 'GNN_layers': ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross'], 'sinkhorn_iterations': 100, 'match_threshold': 0.2, 'cuda': 1, 'name': 'SuperGlueMatcher'}
INFO:root:creating SuperGlue matcher...


Loaded image data: image shape: (378, 504, 3, 55) fx: [378.         504.         416.23374465]
Loaded ../data/trex 14.611281520672364 128.97241851887296
Data:
(55, 3, 5) (55, 378, 504, 3) (55, 2)
HOLDOUT view is 20
PinholeCameraModel: fx: 416.23, fy: 416.23, cx: 252.00, cy: 189.00, width: 504, height: 378
Loaded SuperPoint model


INFO:root:Computing baseline pose and reconstructing points
INFO:root:init_view_matches: 697
INFO:root:fundamental matrix inliers: 258
INFO:root:Computed essential matrix
INFO:root:Choosing correct pose out of 4 solutions


Loaded SuperGlue model ("indoor" weights)


INFO:root:Mean reprojection error for 0 view is 2.771514
INFO:root:Mean reprojection error for 1 view is 2.793250
INFO:root:Mean reprojection error for 2 view is 1.178015
INFO:root:Mean reprojection error for 3 view is 1.138933
INFO:root:Mean reprojection error for 4 view is 1.288387
INFO:root:Mean reprojection error for 5 view is 1.849150
INFO:root:Mean reprojection error for 6 view is 0.606225
INFO:root:Mean reprojection error for 7 view is 0.762643
INFO:root:Mean reprojection error for 8 view is 1.970677
INFO:root:Mean reprojection error for 9 view is 1.202191
INFO:root:Mean reprojection error for 10 view is 0.630305
INFO:root:Mean reprojection error for 11 view is 0.390746
INFO:root:Mean reprojection error for 12 view is 0.635428
INFO:root:Mean reprojection error for 13 view is 0.904894
INFO:root:Mean reprojection error for 14 view is 0.729251
/home/hkust/anaconda3/envs/hloc/lib/python3.8/site-packages/skimage/measure/fit.py:672: RuntimeWarning: divide by zero encountered in scalar

[ 3.30863042  2.515125   -0.78885873]


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [6]:
# bundle adjustment
pipeline.bundle_adjustment()

# visualize optimization results
ipv_prepare(ipv)

Tcw_lst = []
for i, view in enumerate(pipeline.reconstructed_views):
    Tcw_lst.append(view.Tcw)
points_lst = pipeline.reconstructed_map_points

ipv_visualizer = iPV3DVisualizer()
ipv_visualizer.add_trajectory(Tcw_lst)
ipv_visualizer.add_point_cloud(points_lst, pt_size=8.0)
ipv_visualizer.show()

iteration= 0	 chi2= 339336.590165	 time= 0.0245627	 cumTime= 0.0245627	 edges= 17803	 schur= 1	 lambda= 497.336304	 levenbergIter= 1
iteration= 1	 chi2= 282193.672127	 time= 0.0102055	 cumTime= 0.0347682	 edges= 17803	 schur= 1	 lambda= 165.778768	 levenbergIter= 1
iteration= 2	 chi2= 253477.873637	 time= 0.0111415	 cumTime= 0.0459097	 edges= 17803	 schur= 1	 lambda= 55.259589	 levenbergIter= 1
iteration= 3	 chi2= 237541.835764	 time= 0.0107121	 cumTime= 0.0566218	 edges= 17803	 schur= 1	 lambda= 18.419863	 levenbergIter= 1
iteration= 4	 chi2= 225480.202881	 time= 0.0104176	 cumTime= 0.0670394	 edges= 17803	 schur= 1	 lambda= 6.139954	 levenbergIter= 1
iteration= 5	 chi2= 212838.466692	 time= 0.0114856	 cumTime= 0.078525	 edges= 17803	 schur= 1	 lambda= 2.046651	 levenbergIter= 1
iteration= 6	 chi2= 198380.830046	 time= 0.0110939	 cumTime= 0.0896189	 edges= 17803	 schur= 1	 lambda= 0.682217	 levenbergIter= 1
iteration= 7	 chi2= 183184.877462	 time= 0.0111108	 cumTime= 0.10073	 edges= 1

[IncrementalReconstructor] Bundle Adjustment, before_err: 112017.78, after_err: 40011.27
[3.78139915 2.01472292 0.02951826]


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…